In [1]:
import argparse
from argparse import ArgumentParser
from pretrain import T5FineTuner, LoggingCallback,set_seed
import pytorch_lightning as pl
import easydict
from transformers import AutoTokenizer
from torchinfo import summary
import torch

args = easydict.EasyDict({
        "max_input_length": 512,
        "max_output_length": 512,
        "num_train_epochs":100,
        "output_dir": 't5_pretraining',
        "train_batch_size": 2,
        "learning_rate": 1e-5,
        "model_name_or_path":'lcw99/t5-base-korean-text-summary',
        "tokenizer_name_or_path":'lcw99/t5-base-korean-text-summary',
        "freeze_encoder":False,
        "freeze_embeds":False,
        'weight_decay':0.0,
        'adam_epsilon':1e-8,
        'warmup_steps':0,
        'train_batch_size':2,
        'eval_batch_size':2,
        'num_train_epochs':100,
        'gradient_accumulation_steps':1,
        'n_gpu':1,
        'resume_from_checkpoint':None, 
        # 'val_check_interval' : 10,
        'check_val_every_n_epoch':2,
        'n_val':4,
        'val_percent_check': 5,
        'n_train':250,
        'n_test':-1,
        'early_stop_callback':False,
        'fp_16':False, # if you want to enable 16-bit training then install apex and set this to true
        'opt_level':'O1', # you can find out more on optimisation levels here https://nvidia.github.io/apex/amp.html#opt-levels-and-properties
        'max_grad_norm':0.5, # if you enable 16-bit training then set this to a sensible value, 0.5 is a good default
        'seed':42,

})

## Define Checkpoint function
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath=args.output_dir,filename='modelcheckpoint')

## If resuming from checkpoint, add an arg resume_from_checkpoint
train_params = dict(
    accumulate_grad_batches=args.gradient_accumulation_steps,
    accelerator="gpu",
    inference_mode=False,
    # gpus=args.n_gpu,
    max_epochs=args.num_train_epochs,
    precision= 16 if args.fp_16 else 32,
    check_val_every_n_epoch=args.check_val_every_n_epoch,
    callbacks=[LoggingCallback(),checkpoint_callback]
)

set_seed(42)
model = T5FineTuner(args)
tokenizer=AutoTokenizer.from_pretrained(args.tokenizer_name_or_path)
print(summary(model,input_data={'input_ids': torch.tensor([[111]*2048]),
'decoder_input_ids': torch.tensor([[111]*1]),
'attention_mask': torch.tensor([[1]*2048]),
'decoder_attention_mask': torch.tensor([[1]*1]),
}))
trainer = pl.Trainer(**train_params)
trainer.fit(model)

/opt/homebrew/Caskroom/miniforge/base/envs/torch/lib/python3.9/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/homebrew/Caskroom/miniforge/base/envs/torch/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/homebrew/Caskroom/miniforge/base/envs/torch/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


{'max_input_length': 512, 'max_output_length': 512, 'num_train_epochs': 100, 'output_dir': 't5_pretraining', 'train_batch_size': 2, 'learning_rate': 1e-05, 'model_name_or_path': 'lcw99/t5-base-korean-text-summary', 'tokenizer_name_or_path': 'lcw99/t5-base-korean-text-summary', 'freeze_encoder': False, 'freeze_embeds': False, 'weight_decay': 0.0, 'adam_epsilon': 1e-08, 'warmup_steps': 0, 'eval_batch_size': 2, 'gradient_accumulation_steps': 1, 'n_gpu': 1, 'resume_from_checkpoint': None, 'check_val_every_n_epoch': 2, 'n_val': 4, 'val_percent_check': 5, 'n_train': 250, 'n_test': -1, 'early_stop_callback': False, 'fp_16': False, 'opt_level': 'O1', 'max_grad_norm': 0.5, 'seed': 42}


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Layer (type:depth-idx)                                            Output Shape              Param #
T5FineTuner                                                       [1, 2048, 768]            --
├─T5ForConditionalGeneration: 1-1                                 [1, 2048, 768]            --
│    └─T5Stack: 2-1                                               [1, 2048, 768]            123,629,184
│    └─T5Stack: 2-2                                               --                        (recursive)
│    │    └─Embedding: 3-1                                        [1, 2048, 768]            38,674,944
│    └─T5Stack: 2-3                                               --                        (recursive)
│    │    └─Dropout: 3-2                                          [1, 2048, 768]            --
│    │    └─ModuleList: 3-3                                       --                        84,953,472
│    │    └─T5LayerNorm: 3-4                                      [1, 2048, 768]            768
│

/opt/homebrew/Caskroom/miniforge/base/envs/torch/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:615: UserWarning: Checkpoint directory /Users/dongunyun/study/datascience/encoder_decoder/Pretraining_T5_custom_dataset/t5_pretraining exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 275 M 
-----------------------------------------------------
275 M     Trainable params
0         Non-trainable params
275 M     Total params
1,102.317 Total estimated model params size (MB)


...

['a transporter accomplishing  expressed in inner  or deoxycorticosterone (DOCA).  tissues. METHODS: The  utilized RT-qPCR and  and protein abundance  Slc26a4 in murine  and following subcutaneous  100 mg/kg DOCA.', 't(11;15)(q23;q15) in a  carcinogenesis involving multiple  to germline mutations  in the CHEK2  cancer at age  also developed early-onset  bilateral breast cancer,  cell line revealed  t(11;15)(q23;q15). This translocation  in a primary  patient or her  although a nephew']
['SLC26A4 is a disorder of pendrin expression into kidney and heart tissues.', 'LFS is characterized by early-onset tumor types and shows autosomal dominant inheritance for a Li-Fraumeni syndrome family']
['autosomal dominant, human  a key role  a vertebrate system  genetic screens and  a forward genetic  some LFS mutations,  loss-of-function allele with  in vivo. Additionally,  is evolutionarily conserved  novel modifier genes  therapeutic agents for', 'been established as  activity is limited  acquired

/Users/dongunyun/study/datascience/encoder_decoder/Pretraining_T5_custom_dataset/pretrain.py:419: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ds = ds.append(ds2)
/opt/homebrew/Caskroom/miniforge/base/envs/torch/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

tensor(5.5762, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(1.6635, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(2.8314, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(1.7758, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(2.0593, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(1.5923, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(2.1008, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.9025, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(1.9038, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(1.5786, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(1.4628, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(1.7163, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(1.4930, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(1.1504, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(1.0087, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(1.2622, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(1.3853, device='mps:0', grad_fn=<NllLossBackward0

/opt/homebrew/Caskroom/miniforge/base/envs/torch/lib/python3.9/site-packages/pytorch_lightning/trainer/call.py:52: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
